In [49]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

URL = "https://mercados.larrainvial.com/www/v2/index.html?mercado=chile"
def _nearest_title_js():
    return r"""
const tables = Array.from(document.querySelectorAll('table'));
function getNearestTitle(el) {
  let cont = el.closest(".card, .panel, .box, section, .widget, .portlet, .modulo, .module, .contenedor, .container") || el.parentElement;
  function getTitleFrom(node) {
    if (!node) return "";
    const pick = node.querySelectorAll("h1,h2,h3,h4,h5,.title,.titulo,.header,.card-header,.panel-heading,.box-title,.widget-title,.modulo-titulo, .section-title, .section-header");
    let text = "";
    for (const t of pick) {
      const s = (t.innerText || "").trim();
      if (s && s.length > 0) text += " " + s;
    }
    if (text.trim().length > 0) return text.trim();
    return "";
  }
  let title = getTitleFrom(cont);
  if (!title) {
    let sib = el.previousElementSibling;
    while (sib) {
      const s = (sib.innerText || "").trim();
      if (s && s.length > 0) { title = s; break; }
      sib = sib.previousElementSibling;
    }
  }
  if (!title) {
    let sib2 = cont ? cont.previousElementSibling : null;
    while (sib2) {
      const s = (sib2.innerText || "").trim();
      if (s && s.length > 0) { title = s; break; }
      sib2 = sib2.previousElementSibling;
    }
  }
  return (title || "").trim();
}

return tables.map((t, i) => {
  return {
    idx: i,
    title: getNearestTitle(t),
    outerHTML: t.outerHTML
  };
});
"""
def get_todas_las_tablas(timeout: int = 45,
                         headless: bool = True,
                         debug: bool = True) -> dict:
    opts = webdriver.ChromeOptions()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--window-size=1920,1080")
    opts.add_argument("--log-level=3")
    opts.add_experimental_option("excludeSwitches", ["enable-logging","enable-automation"])
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts) 
    text = []
    try:
        wait = WebDriverWait(driver, timeout)
        driver.get(URL)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        for y in (300, 900, 1500, 2200, 0):
            driver.execute_script(f"window.scrollTo({{top:{y}}});")
            time.sleep(0.6)
        tables_info = driver.execute_script(_nearest_title_js())
        if debug:
            for t in tables_info:
                text.append(t)
    finally:
        driver.quit()
    return text
if __name__ == "__main__":
    tablas = get_todas_las_tablas(headless=True, debug=True)
    title_text = tablas[2]['title']
    lineas = [l for l in title_text.strip().split("\n") if l.strip()]
    _ = lineas[0] 
    filas = [l.split("\t") for l in lineas[1:]]
    df = pd.DataFrame(filas, columns=["Acción", "Precio", "Var.%"])
    def to_float_ch(s):
      s = s.strip()
      s = s.replace(".", "")
      s = s.replace(",", ".")         
      s = s.replace("%", "")           
      s = s.replace("+", "")           
      return float(s)
    df["Precio"] = df["Precio"].apply(to_float_ch)
    df["Var.%"] = df["Var.%"].apply(to_float_ch)
    tickers = [
    "AGUAS-A","ANDINA-B","BCI","BSANTANDER","CAP","CCU","CENCOSUD","CHILE",
    "CMPC","COLBUN","CONCHATORO","COPEC","ECL","ENELAM","ENELCHILE","ENELGXCH",
    "ENTEL","FALABELLA","FORUS","HITES","ILC","ITAUCL","LTM","MALLPLAZA",
    "PARAUCO","RIPLEY","SALFACORP","SECURITY","SMU","SONDA","SQM-B","TRICOT","VAPORES"]
    df = df[df["Acción"].isin(tickers)]
    df = df.set_index("Acción").reindex(tickers).reset_index()
    print(df)

        Acción    Precio  Var.%
0      AGUAS-A    338.38   0.11
1     ANDINA-B   3858.40   1.24
2          BCI  39085.00   0.73
3   BSANTANDER     57.50   0.00
4          CAP   4987.00   0.85
5          CCU   5920.00   0.85
6     CENCOSUD   3088.00  -0.39
7        CHILE    137.69  -0.08
8         CMPC   1521.60   1.44
9       COLBUN    144.59   0.41
10  CONCHATORO   1106.30  -0.33
11       COPEC   7140.60   1.29
12         ECL   1190.00  -0.33
13      ENELAM     97.86  -0.65
14   ENELCHILE     68.07   0.12
15    ENELGXCH    444.00   1.58
16       ENTEL   3276.00   0.61
17   FALABELLA   5600.00   0.36
18       FORUS   2070.60   1.01
19       HITES    115.00   4.58
20         ILC  12540.00   1.70
21      ITAUCL  13809.00   0.98
22         LTM     23.63  -0.71
23   MALLPLAZA   2266.40   1.17
24     PARAUCO   2189.00   1.34
25      RIPLEY    457.40   1.64
26   SALFACORP    851.84   2.37
27    SECURITY    291.02   0.06
28         SMU    163.86   0.96
29       SONDA    338.23  -0.74
30      